In [2]:
from Data import *
from NeuralNetwork import *
%load_ext autoreload
%autoreload 2

In [6]:
(train_images, train_labels), (test_images, test_labels)   =  load_data(name = 'fashion_mnist')
train_images, train_labels, valid_images, valid_labels = split_validation(train_images, train_labels, valid_size=0.1, seed = 42)
train_images, train_labels,_,_ = split_validation(train_images, train_labels, valid_size=0.95, seed = 42)
X_train, X_valid, X_test = preprocess(train_images), preprocess(valid_images), preprocess(test_images)
y_train, y_valid, y_test = one_hot_encoded(train_labels), one_hot_encoded(valid_labels), one_hot_encoded(test_labels)

#Reading the config file

#Getting the configutation from wandb sweep
num_layers = 4
hidden_size = 252
lr = 1e-4
beta = 0.99
beta_m = 0.99
beta_v = 0.99
alpha = 0.5
winit = "Xavier"
bs = 32
epochs = 3
optimizer_name = 'nadam'
g_activation = "tanh"


#Preparing hyperparameters for our optimizer
if optimizer_name == 'sgd':
    optim_hp = {'learning_rate':lr, 'alpha': alpha}
elif optimizer_name == 'momentum' or optimizer_name == 'nag' or optimizer_name == 'rmsprop':
    optim_hp = {'learning_rate':lr, 'beta': beta, 'alpha': alpha}
elif optimizer_name == 'adam' or optimizer_name == 'nadam':
    optim_hp = {'learning_rate':lr, 'beta_m': beta_m, 'beta_v': beta_v, 'alpha':alpha}

n_features = X_train.shape[1]
n_classes = y_train.shape[1]
hidden_sizes = [hidden_size]*num_layers + [n_classes]

network_hp = {'batch_size': bs, 'g_activation': ACTIVATIONS_MAP[g_activation]}
mynn = MyNeuralNetwork(n_features, hidden_sizes, n_classes)
mynn.params_init(way = winit)
# mynn.update_network_hp(**network_hp)  
# mynn.update_optim_hp(optimizer_name,**optim_hp)    
# mynn.train(X_train,y_train,X_valid,y_valid,epochs = epochs)

Sum : 9.613776882846402
Sum : -8.085697174227047
Sum : -20.84288965985256
Sum : 18.229967252781897
Sum : 5.693926544576241


In [20]:
y_test_pred = mynn.predict(X_test)

In [21]:
print(np.sum(np.argmax(y_test,axis = 1)==np.argmax(y_test_pred,axis = 1))/y_test.shape[0])

0.7294


In [38]:
from collections import Counter
np.unique(np.argmax(y_train,axis = 1), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([270, 270, 270, 270, 270, 270, 270, 270, 270, 270]))

In [19]:
mynn.weights[0]

array([[ 0.44397905, -0.2270532 ,  0.85351622, ..., -1.66284238,
         0.06621643,  0.46687932],
       [ 0.55918849,  1.16691805,  0.87170058, ..., -0.33291553,
         0.1899259 ,  0.71354788],
       [-1.5601007 , -0.5778982 ,  0.67630795, ..., -0.57130518,
         0.88317859,  0.44534818],
       ...,
       [-0.53291019, -0.67599409,  1.37280076, ...,  1.81105739,
        -0.12194865,  0.84618229],
       [ 0.82133766,  0.64010845, -0.43371267, ..., -1.92825264,
        -0.22334057, -0.67335536],
       [ 1.76370395, -0.02307934, -0.35251759, ...,  1.24726423,
         1.64829953,  1.31408006]])